# Давай попробуем всякие opensource-ные решения вкрутить

In [24]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

LANG_ID = "ru"
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-russian"
SAMPLES = 5

processor = Wav2Vec2Processor.from_pretrained("resources/wav2vec2ctc_processor.pt")
model = Wav2Vec2ForCTC.from_pretrained("resources/wav2vec2ctc.pt")


In [25]:
# model.save_pretrained("resources/wav2vec2ctc.pt")
# processor.save_pretrained("resources/wav2vec2ctc_processor.pt")

In [26]:
example_path = "resources/example.wav"
samples, sr = torchaudio.load(example_path)
print(samples.shape, sr)
# Downsampling to 16k
transform = torchaudio.transforms.Resample(
    orig_freq=sr, new_freq=16_000
)
samples = transform(samples).squeeze(0)
sr = 16_000
print(samples.shape, sr)


torch.Size([1, 101376]) 32000
torch.Size([50688]) 16000


In [27]:
inputs = processor(samples, sampling_rate=sr, return_tensors="pt", padding=True)
print(inputs["input_values"].shape)

with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)
predicted_sentences = processor.batch_decode(predicted_ids)
predicted_sentences

torch.Size([1, 50688])


['а и правда никакой']

In [28]:
# нихуя себе, работает)